In [1]:
import os

In [2]:
pwd

'd:\\ML Projects\\Chest Cancer Classification\\chest-cancer-classifier\\research'

In [3]:
os.chdir("../")

In [4]:
pwd

'd:\\ML Projects\\Chest Cancer Classification\\chest-cancer-classifier'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/dhavalparmar1108/chest-cancer-classifier.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="dhavalparmar1108"
os.environ["MLFLOW_TRACKING_PASSWORD"]="873f012a48d5ece119b384103231f0bb7149b08c"

In [6]:
import tensorflow as tf

In [7]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [9]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [10]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chest-CT-Scan-data",
            mlflow_uri="https://dagshub.com/dhavalparmar1108/chest-cancer-classifier.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [11]:
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [12]:
from cnnClassifier import logger

In [13]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        print(" re " + mlflow.get_registry_uri())
        print(" tr " + mlflow.get_tracking_uri())
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(
                    sk_model=model,
                    artifact_path="sklearn-model",
                    registered_model_name="sk-learn-random-forest-reg-model",)
            else:
                 mlflow.sklearn.log_model(
                    sk_model=model,
                    artifact_path="sklearn-model")

In [14]:
print("hi")

hi


In [15]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

Found 102 images belonging to 2 classes.


c:\Users\Dhaval\miniconda3\envs\cancer\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


7/7 ━━━━━━━━━━━━━━━━━━━━ 59s 7s/step - accuracy: 1.0000 - loss: 4.6285e-05
 re https://dagshub.com/dhavalparmar1108/chest-cancer-classifier.mlflow
 tr https://dagshub.com/dhavalparmar1108/chest-cancer-classifier.mlflow


c:\Users\Dhaval\miniconda3\envs\cancer\Lib\site-packages\_distutils_hack\__init__.py:32: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Registered model 'sk-learn-random-forest-reg-model' already exists. Creating a new version of this model...
2024/08/23 15:27:06 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: sk-learn-random-forest-reg-model, version 5
Created version '5' of model 'sk-learn-random-forest-reg-model'.
